In [1]:
import pandas as pd
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
import networkx as nx
from scipy.sparse import lil_matrix
import matplotlib.pyplot as plt

from utils import*

In [2]:
data = pd.read_csv('lenta-ru-train.csv')

In [3]:
data

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1
...,...,...,...,...
206632,Бывшую вице-президента Tiffany обвинили в милл...,Бывшей вице-президенту ювелирной компании Tiff...,Экономика,0
206633,Минфин предложил увеличить штрафы за незаконны...,Минфин предлагает увеличить размер штрафа для ...,Экономика,0
206634,Потребительская уверенность россиян вернулась ...,Индекс потребительской уверенности россиян в п...,Экономика,0
206635,Тарпищеву дали американскую визу,Капитан женской сборной России по теннису Шами...,Спорт,1


In [4]:
test = data[0:100]

In [5]:
graph_1 = text2graph(test['text'][0], 0.1)
graph_2 = text2graph(test['text'][1], 0.1)

In [6]:
jaccard_similarity(graph_1, graph_2) # diff between economics and culture topics

0.017

In [7]:
graph_3 = text2graph(test['text'][7], 0.1)
graph_4 = text2graph(test['text'][9], 0.1)

In [8]:
jaccard_similarity(graph_1, graph_3) # diff between economics and economics topics

0.105

In [9]:
average_distance(test, 'Экономика')

0.088

In [10]:
average_distance(test, 'Культура')

0.069

In [11]:
# ideas:
# 1.make 'average' graph for each class
#   make function to diff them

# 2.graph representation of texts --> vector representation
#   distance metric vector representations

# 3.graph distance metric

# 4.compute average distance between graphs inside topics